In [8]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install underthesea

  Obtaining dependency information for underthesea from https://files.pythonhosted.org/packages/c0/26/a2615fb23899ec747eaf7d7b3d4ae6b1959b71ae2a8546e121a8ce375d9c/underthesea-6.8.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
     -------- ------------------------------ 30.7/139.4 kB 1.4 MB/s eta 0:00:01
     ---------- -------------------------- 41.0/139.4 kB 667.8 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/139.4 kB 667.8 kB/s eta 0:00:01
     ---------------- -------------------- 61.4/139.4 kB 409.6 kB/s eta 0:00:01
     ------------------------------- ---- 122.9/139.4 kB 554.9 kB/s eta 0:00:01
     ---------------------------------- - 133.1/139.4 kB 563.7 kB/s eta 0:00:01
     ---------------------------------- - 133.1/139.4 kB 563.7 kB/s eta 0:00:01
     ------------------------------------ 139.4/139.4 kB 414.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - --

In [4]:
!pip install findspark

In [19]:
!pip install textblob

     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
     - ----------------------------------- 30.7/636.8 kB 640.0 kB/s eta 0:00:01
     -- ---------------------------------- 41.0/636.8 kB 388.9 kB/s eta 0:00:02
     ----- ------------------------------- 92.2/636.8 kB 655.4 kB/s eta 0:00:01
     ------ ----------------------------- 122.9/636.8 kB 798.9 kB/s eta 0:00:01
     ------ ----------------------------- 122.9/636.8 kB 798.9 kB/s eta 0:00:01
     ------------ ----------------------- 225.3/636.8 kB 808.4 kB/s eta 0:00:01
     --------------- -------------------- 276.5/636.8 kB 850.6 kB/s eta 0:00:01
     -------------------- --------------- 368.6/636.8 kB 995.6 kB/s eta 0:00:01
     ------------------------ ------------- 409.6/636.8 kB 1.0 MB/s eta 0:00:01
     ----------------------------- -------- 491.5/636.8 kB 1.1 MB/s eta 0:00:01
     --------------------------------- ---- 563.2/636.8 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ - 614

In [39]:
!pip install torch

     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/172.3 MB 1.3 MB/s eta 0:02:15
     -------------------------------------- 0.1/172.3 MB 656.4 kB/s eta 0:04:23
     -------------------------------------- 0.1/172.3 MB 980.4 kB/s eta 0:02:56
     -------------------------------------- 0.2/172.3 MB 952.6 kB/s eta 0:03:01
     ---------------------------------------- 0.2/172.3 MB 1.2 MB/s eta 0:02:23
     ---------------------------------------- 0.3/172.3 MB 1.2 MB/s eta 0:02:25
     ---------------------------------------- 0.4/172.3 MB 1.2 MB/s eta 0:02:22
     ---------------------------------------- 0.5/172.3 MB 1.3 MB/s eta 0:02:09
     ---------------------------------------- 0.6/172.3 MB 1.4 MB/s eta 0:02:04
     ---------------------------------------- 0.6/172.3 MB 1.3 MB/s eta 0:02:12
     ---------------------------------------- 0.7/172.3 MB 1.4 MB/s eta 0:02:02
     ---------------------------------------- 0

In [40]:
!wget "https://docs.google.com/uc?export=download&id=1qsNJ6xmXqLg_xQ51-OONNHjYaERsh5RE&confirm=t" -O vi_word2vec.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import findspark
findspark.init()
import pandas as pd
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Khởi tạo Spark Session
spark = (
    SparkSession.builder.appName("MySparkApp")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

pd.options.display.max_columns = None
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 150
sns.set(color_codes=True)

In [42]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import col, udf
from textblob import TextBlob
from underthesea import sent_tokenize, word_tokenize
# Hàm deEmojify đã chỉnh sửa để loại bỏ emoji và kí tự đặc biệt
def deEmojify(text):
    # Loại bỏ emoji
    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    
    # Loại bỏ kí tự đặc biệt và kí tự xuống dòng
    clean_text = re.sub(r'[^\w\s]+|\r\n', '', text)
    clean_text = re.sub(r'http\S+', '', clean_text)
    # Loại bỏ emoji từ văn bản
    text_without_emoji = re.sub(emoj, '', clean_text)
    
    return text_without_emoji

# Define a UDF to analyze sentiment
def analyze_sentiment(comment):
   sentiment_result = sentiment(comment)
   return sentiment_result

def analyze_sentiment_vietnamese(text):
    # Sử dụng underthesea để tách câu và từ
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]
    print(words)
    # Đoạn code này để tính điểm cảm xúc dựa trên từng từ và câu,
    # bạn có thể thay đổi logic ở đây để phù hợp với nhu cầu của bạn
    sentiment_score = 0
    for sentence_words in words:
        sentence_blob = TextBlob(" ".join(sentence_words))
        sentence_sentiment = sentence_blob.sentiment.polarity
        sentiment_score += sentence_sentiment
        print(sentiment_score)
    
    if sentiment_score > 0:
        return "Positive"
    elif sentiment_score < 0:
        return "Negative"
    else:
        return "Neutral"
    


In [37]:
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from underthesea import sent_tokenize
from pyspark.sql.functions import col


timestampformat = "EEE MMM dd HH:mm:ss zzz yyyy"
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# Định nghĩa schema cho DataFrame
schema = StructType([
    StructField("author", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("video_id", StringType(), True),
    StructField("playlist_program", StringType(), True)
])

# Đường dẫn tới file JSON chứa dữ liệu comment
IN_PATH_RAW = "../data/commentvideo_export.json"

# Tạo DataFrame từ file JSON và áp dụng schema và multiline option
df_train_raw = spark.read.schema(schema).option('multiline', True).json(IN_PATH_RAW)

# Định nghĩa UDF để loại bỏ emoji và xử lý dữ liệu comment
emojify_udf = udf(deEmojify, StringType())

# Áp dụng UDF để làm sạch comment
df_cleaned = df_train_raw.withColumn("comment", emojify_udf(df_train_raw["comment"]))

# Tạo một cột mới "sentiment" bằng cách áp dụng UDF analyze_sentiment
sentiment_udf = udf(analyze_sentiment_vietnamese, StringType())
df_with_sentiment = df_cleaned.withColumn("sentiment", sentiment_udf(df_cleaned["comment"]))

# Loại bỏ các dòng có comment là null hoặc rỗng
df_with_sentiment = df_with_sentiment.filter((col("comment").isNotNull()) & (col("comment") != ""))

# Hiển thị các cột cần thiết
df_with_sentiment.select("author", "comment", "sentiment", "playlist_program").show(truncate=False)



+------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+----------------+
|author                  |comment                                                                                                                                                                                                                                                                             |sentiment|playlist_program|
+------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+----------------+
|Vie Ch

In [43]:
text = "Mê team anh Baus quá huhu Hy vọng anh trở lại vào rap Việt mùa 4"

print(analyze_sentiment_vietnamese(text))

[['Mê', 'team', 'anh', 'Baus', 'quá', 'huhu', 'Hy vọng', 'anh', 'trở lại', 'vào', 'rap', 'Việt', 'mùa', '4']]
0.0
Neutral


In [72]:

spark.stop()

root
 |-- author;comment;video_id;playlist_program: string (nullable = true)

